# Цели
Здесь я собираюсь сделать модель для предсказания цены закрытия акции по информации за $N$ предыдущих дней. 
А так же проверить следущие гипотезы:
1. Рынок приспосабливается к стратегиям. Иными словами, чем дальше период на котором обучалась модель от цели предсказания, тем ниже точность предсказания.
2. Внешние данны важны. Иными словами, предсказания котировок с ипользованием внешних текстов получаются точнее, чем без них.

В роли внешних текстов используются [новости с MOEX](https://www.moex.com/ru/news/?ncat=106). А точнее, их заголовки.

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer

# Обработка данных   
## Исходные данные
Здесь мы собираем данные как есть

### Акции

In [3]:
path_to_tools = 'data/tools/list_tools.txt'
df_tools = pd.read_csv(path_to_tools,sep='\t')
df_tools.head()

,SECID,BOARDID,SHORTNAME,PREVPRICE,LOTSIZE,FACEVALUE,STATUS,BOARDNAME,DECIMALS,SECNAME,...,PREVDATE,ISSUESIZE,ISIN,LATNAME,REGNUMBER,PREVLEGALCLOSEPRICE,CURRENCYID,SECTYPE,LISTLEVEL,SETTLEDATE
0,ABIO,TQBR,iАРТГЕН ао,73.50,10,0.10,A,Т+: Акции и ДР - безадрес.,2,"ПАО ""Артген""",...,2024-12-12,92645451,RU000A0JNAB6,ARTGEN ao,1-01-08902-A,74.420,SUR,1,2,2024-12-16
1,ABRD,TQBR,АбрауДюрсо,165.00,10,1.00,A,Т+: Акции и ДР - безадрес.,1,Абрау-Дюрсо ПАО ао,...,2024-12-12,98000184,RU000A0JS5T7,Abrau-Durso ao,1-02-12500-A,165.000,SUR,1,3,2024-12-16
2,ACKO,TQBR,АСКО ао,3.58,100,1.00,A,Т+: Акции и ДР - безадрес.,2,АСКО ПАО ао,...,2024-12-12,536000000,RU000A0JXS91,ASKO ao,1-01-52065-Z,3.580,SUR,1,3,2024-12-16
3,AFKS,TQBR,Система ао,11.93,100,0.09,A,Т+: Акции и ДР - безадрес.,3,"АФК ""Система"" ПАО ао",...,2024-12-12,9650000000,RU000A0DQZE3,AFK Sistema,1-05-01669-A,11.932,SUR,1,1,2024-12-16
4,AFLT,TQBR,Аэрофлот,48.54,10,1.00,A,Т+: Акции и ДР - безадрес.,2,Аэрофлот-росс.авиалин(ПАО)ао,...,2024-12-12,3975771215,RU0009062285,Aeroflot,1-01-00010-A,48.550,SUR,1,1,2024-12-16



### Свечи

In [4]:
path_to_all_tools_candles = 'data/candles'
dfs = []
for path_to_candles in Path(path_to_all_tools_candles).glob('*'):
    df_candles = pd.read_csv(path_to_candles,sep='\t')
    df_candles['SECID'] = path_to_candles.name[:-6]
    dfs.append(df_candles)
df_candles = pd.concat(dfs)
df_candles.head()

C:\Users\user\AppData\Local\Temp\ipykernel_15600\2080516023.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_candles = pd.concat(dfs)


,begin,end,open,high,low,close,value,volume,SECID
0,2014-06-09 00:00:00,2014-06-09 23:59:59,15.508,15.508,14.801,15.000,22579.4,1500.0,ABIO
1,2014-06-10 00:00:00,2014-06-10 23:59:59,14.871,15.153,14.871,14.881,25469.9,1700.0,ABIO
2,2014-06-11 00:00:00,2014-06-11 23:59:59,15.000,15.000,14.999,14.999,2999.9,200.0,ABIO
3,2014-06-16 00:00:00,2014-06-16 23:59:59,15.468,15.468,15.000,15.000,24522.3,1600.0,ABIO
4,2014-06-17 00:00:00,2014-06-17 23:59:59,15.100,15.120,15.100,15.120,738392.0,48900.0,ABIO



### Текст

In [5]:
path_to_text = 'data/texts/moex_title.txt'
df_text = pd.read_csv(path_to_text)
df_text.head()

,id,tag,title,published_at,modified_at
0,75803,site,"О внесении изменений в Список ценных бумаг, до...",2024-12-13 17:34:03,2024-12-13 14:34:05
1,75802,site,"Цена исполнения фьючерсов на сахар, декабрь 2024",2024-12-13 17:30:45,2024-12-13 17:30:45
2,75801,site,Дополнительные условия проведения торгов отдел...,2024-12-13 17:25:56,2024-12-13 17:25:56
3,75799,site,"13.12.2024, 17-24 (мск) изменены значения нижн...",2024-12-13 17:24:00,2024-12-13 17:25:10
4,75797,site,"13.12.2024, 17-16 (мск) изменены значения верх...",2024-12-13 17:16:09,2024-12-13 17:20:04


## Входные данные
Здесь мы приводим данные к универсальному виду, который и будем в дальнейшем передавать в модель для дальнейшей предобработки

### Акционные данные

In [6]:
df_tools = df_tools[['SECID','BOARDID','SHORTNAME','SECNAME','LOTSIZE','SECTYPE','LISTLEVEL']]
df_tools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SECID      260 non-null    object
 1   BOARDID    260 non-null    object
 2   SHORTNAME  260 non-null    object
 3   SECNAME    260 non-null    object
 4   LOTSIZE    260 non-null    int64 
 5   SECTYPE    260 non-null    object
 6   LISTLEVEL  260 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 14.3+ KB


In [7]:
df_candles['day_date'] = df_candles['begin'].map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d')) 
df_candles = df_candles[['SECID','day_date','close','value','volume']]
df_candles.head()

,SECID,day_date,close,value,volume
0,ABIO,2014-06-09,15.000,22579.4,1500.0
1,ABIO,2014-06-10,14.881,25469.9,1700.0
2,ABIO,2014-06-11,14.999,2999.9,200.0
3,ABIO,2014-06-16,15.000,24522.3,1600.0
4,ABIO,2014-06-17,15.120,738392.0,48900.0


In [8]:
df_market = df_tools.merge(df_candles,on='SECID')
df_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525620 entries, 0 to 525619
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   SECID      525620 non-null  object        
 1   BOARDID    525620 non-null  object        
 2   SHORTNAME  525620 non-null  object        
 3   SECNAME    525620 non-null  object        
 4   LOTSIZE    525620 non-null  int64         
 5   SECTYPE    525620 non-null  object        
 6   LISTLEVEL  525620 non-null  int64         
 7   day_date   525620 non-null  datetime64[ns]
 8   close      525620 non-null  float64       
 9   value      525620 non-null  float64       
 10  volume     525620 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 44.1+ MB


### Текстовые данные

In [9]:
df_text['day_date'] = df_text['modified_at'].map(lambda x: datetime.strptime(x[:10],'%Y-%m-%d')) 
df_text = df_text[['day_date','title']].rename(columns={'title':'text'})
df_text = df_text.groupby('day_date',as_index=False)['text'].agg('\n'.join)
df_text.head()

,day_date,text
0,2011-12-17,О конфигурации Фондового рынка Группы ММВБ-РТС...
1,2011-12-29,Об установлении новых условий заключения сдело...
2,2012-01-12,Итоги рынков Группы ММВБ-РТС с 3 по 6 января 2...
3,2012-01-17,Итоги рынков Группы ММВБ-РТС за неделю с 9 по ...
4,2012-01-18,Об установлении новых условий заключения сдело...


## Модель-съедобный вид
Здесь мы превращаем входные данные, в данные которые пойдут непосредственно в регрессоры. А именно, в таблицу со столбцами:
* Целевой инструмент
    * Ид
    * Доп. инфо
    * 30 дней свечей (по отдельному столбцу на день)
* Весь рынок
    * Среднее
    * Дисперсия
    * Квантили
* Текст
    * 30 дней векторов новостей с рынка или с рбк (по отдельному столбцу на день)
* Дата перед предсказываемым днем
    * Год
    * Месяц
    * День месяца
    * День недели

### Обрезка входных данных по 30 дней

In [10]:
TIME_DELTA = 30
td = df_market['day_date'].max()
fd = td - timedelta(TIME_DELTA)
df_day_delta = pd.DataFrame([i for i in range(TIME_DELTA+1)],columns=['day_delta'])
# df_day_delta

In [11]:
df_market_30_day = df_market[(df_market['day_date'] <= td) & (df_market['day_date'] >= fd)].copy()
df_market_30_day['day_delta'] = (td - df_market_30_day['day_date']).map(lambda x: x.days)
del df_market_30_day['day_date']
df_market_30_day = df_day_delta.merge(df_market_30_day,how='left',on='day_delta')
df_market_30_day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5625 entries, 0 to 5624
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   day_delta  5625 non-null   int64  
 1   SECID      5617 non-null   object 
 2   BOARDID    5617 non-null   object 
 3   SHORTNAME  5617 non-null   object 
 4   SECNAME    5617 non-null   object 
 5   LOTSIZE    5617 non-null   float64
 6   SECTYPE    5617 non-null   object 
 7   LISTLEVEL  5617 non-null   float64
 8   close      5617 non-null   float64
 9   value      5617 non-null   float64
 10  volume     5617 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 483.5+ KB


In [12]:
df_text_30_day = df_text[(df_text['day_date'] <= td) & (df_text['day_date'] >= fd)].copy()
df_text_30_day['day_delta'] = (td - df_text_30_day['day_date']).map(lambda x: x.days)
del df_text_30_day['day_date']
df_text_30_day = df_day_delta.merge(df_text_30_day,how='left',on='day_delta')
df_text_30_day[df_text_30_day.isna()] = ''
df_text_30_day

,day_delta,text
0,0,"О внесении изменений в Список ценных бумаг, до..."
1,1,Итоги выпуска биржевых облигаций\nО регистраци...
2,2,"О регистрации выпуска биржевых облигаций, о вк..."
3,3,Дополнительные условия проведения торгов облиг...
4,4,Об обновлении параметров индексов акций\nОб ус...
5,5,
6,6,О начале торгов ценными бумагами &quot;9&quot;...
7,7,О дополнительных условиях предоставления досту...
8,8,Итоги выпуска биржевых облигаций\nО регистраци...
9,9,Об изменении риск-параметров на фондовом рынке...


### Целевой инструмент

In [23]:
# ИД и пр. инфо
df_result = df_market_30_day[[
    'SECID'
    ,'BOARDID'
    ,'SHORTNAME'
    ,'SECNAME'
    ,'LOTSIZE'
    ,'SECTYPE'
    ,'LISTLEVEL'
]].drop_duplicates()

In [25]:
# 30 дней свечей
df_day_pivot_candles = pd.pivot_table(df_market_30_day,values=['close','value','volume'],columns='day_delta',index=['SECID'],fill_value=-1)
df_day_pivot_candles.columns = ['_'.join(map(lambda x: str(x),col)) for col in df_day_pivot_candles.columns]
df_day_pivot_candles.reset_index()


df_result = df_result.merge(df_day_pivot_candles,on='SECID')
df_result.columns = ['tg_tls_' + col for col in df_result.columns]
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Columns: 145 entries, tg_tls_SECID to tg_tls_volume_30
dtypes: float64(140), object(5)
memory usage: 285.6+ KB


### Весь рынок
* Среднее
* Дисперсия
* Квантили

In [26]:
quantile_25 = lambda x: np.quantile(x,.25)
quantile_50 = lambda x: np.quantile(x,.5)
quantile_75 = lambda x: np.quantile(x,.75)
df_market_statistic = pd.pivot_table(
    df_market_30_day
    ,values=['close','value','volume']
    ,columns='day_delta'
    ,index=[]
    ,aggfunc=[
        'mean'
        ,'std'
        ,quantile_25
        ,quantile_50
        ,quantile_75
    ]
)
df_market_statistic = df_market_statistic.unstack().to_frame().T
df_market_statistic.columns = ['_'.join(map(lambda x: str(x),col)) for col in df_market_statistic.columns]

df_market_statistic.columns = ['all_mrk_' + col for col in df_market_statistic.columns]

df_result['key'] = df_market_statistic['key'] = 0 
df_result = df_result.merge(df_market_statistic,on='key').drop('key',axis=1)
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Columns: 490 entries, tg_tls_SECID to all_mrk_<lambda>_30_volume
dtypes: float64(485), object(5)
memory usage: 964.8+ KB


### Текст
* 30 дней векторов новостей с рынка ~~или с рбк~~ (по отдельному столбцу на день)

In [29]:
vectorizer = TfidfVectorizer()
vectorize_text = vectorizer.fit_transform(df_text_30_day['text'])
vectorize_text = vectorize_text.reshape((-1,np.prod(vectorize_text.shape)))
vectorize_text = vectorize_text.toarray()

vectorize_text = pd.DataFrame(vectorize_text)
vectorize_text.columns = ['txt_' + str(col) for col in vectorize_text.columns]

df_result['key'] = vectorize_text['key'] = 0
df_result = df_result.merge(vectorize_text,on='key').drop('key',axis=1)
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Columns: 27801 entries, tg_tls_SECID to txt_27310
dtypes: float64(27796), object(5)
memory usage: 53.5+ MB


### Дата перед предсказываемым днем
* Год
* Месяц
* День месяца
* День недели

In [ ]:
df_result['pred_year'] = td.year
df_result['pred_month'] = td.month
df_result['pred_day'] = td.day
df_result['pred_day_of_week'] = td.day_of_week

df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Columns: 27736 entries, SECID to pred_day_of_week
dtypes: float64(27727), int64(4), object(5)
memory usage: 53.3+ MB


In [20]:
df_result

,SECID,BOARDID,SHORTNAME,SECNAME,LOTSIZE,SECTYPE,LISTLEVEL,close_0,close_1,close_2,...,27305,27306,27307,27308,27309,27310,pred_year,pred_month,pred_day,pred_day_of_week
0,ABIO,TQBR,iАРТГЕН ао,"ПАО ""Артген""",10.0,1,2.0,73.120,73.50,75.120,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4
1,ABRD,TQBR,АбрауДюрсо,Абрау-Дюрсо ПАО ао,10.0,1,3.0,165.000,165.00,166.000,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4
2,AFKS,TQBR,Система ао,"АФК ""Система"" ПАО ао",100.0,1,1.0,11.964,11.93,12.149,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4
3,AFLT,TQBR,Аэрофлот,Аэрофлот-росс.авиалин(ПАО)ао,10.0,1,1.0,48.130,48.54,49.970,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4
4,AKRN,TQBR,Акрон,Акрон ПАО ао,1.0,1,2.0,17068.000,17006.00,17186.000,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,KRKN,TQBR,СаратНПЗ,Саратовский НПЗ ПАО ао,1.0,1,3.0,-1.000,8350.00,8550.000,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4
248,VSYDP,TQBR,ВыбСудЗ ап,Выборгский Суд.Завод ПАО ап,1.0,2,3.0,-1.000,7750.00,7850.000,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4
249,KOGK,TQBR,КоршГОК ао,Коршуновский ГОК ПАО ао,1.0,1,3.0,-1.000,-1.00,34000.000,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4
250,PRMB,TQBR,Приморье,"АКБ ""Приморье"" ао",1.0,1,3.0,-1.000,-1.00,27000.000,...,0.0,0.0,0.0,0.0,0.0,0.0,2024,12,13,4


# Создание данных